해운/조선 지수

해운/조선 지수이며, 발틱 건화물 운임 지수(BDI), 상하이 컨테이너 운임 지수(SCFI), 클락슨 신조선가 지수가 있어요

1. BDI(발틱 건화물 운임 지수)

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
import time
from datetime import datetime, timedelta

# 크롬 옵션 설정 (headless 모드)
options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('log-level=3')

# 크롬 드라이버 경로 지정 (환경에 맞게 변경)
driver = webdriver.Chrome(options=options)  # 또는 webdriver.Chrome('chromedriver경로', options=options)

# 타겟 URL
url = 'https://www.investing.com/indices/baltic-dry-historical-data'

driver.get(url)
time.sleep(5)  # 페이지 로딩 대기

all_data = []

while True:
    html = driver.page_source
    dfs = pd.read_html(html)
    page_data = dfs[0]
    all_data.append(page_data)
    
    try:
        next_button = driver.find_element(By.XPATH, "//a[@title='Next']")
        if 'disabled' in next_button.get_attribute('class'):
            break  # 다음 페이지 없으면 종료
        next_button.click()
        time.sleep(3)  # 페이지 로딩 대기
    except:
        break  # Next 버튼 없으면 종료

driver.quit()

# 모든 페이지 데이터 합치기
full_data = pd.concat(all_data, ignore_index=True)

# 날짜 컬럼 datetime 변환
full_data['Date'] = pd.to_datetime(full_data['Date'])

# 최신 날짜 기준 2달 전 날짜 계산
max_date = full_data['Date'].max()
cutoff_date = max_date - pd.DateOffset(months=2)

# 2달치 데이터 필터링
bdi_recent = full_data[full_data['Date'] >= cutoff_date]

# 날짜, 종가(Price) 컬럼만 추출
bdi_recent = bdi_recent[['Date', 'Price']]

# 컬럼명 한글로 변경
bdi_recent.columns = ['날짜', '종가']

# 날짜 오름차순 정렬
bdi_recent = bdi_recent.sort_values(by='날짜').reset_index(drop=True)

bdi_recent

C:\Users\mcdor\AppData\Local\Temp\ipykernel_20532\3646955635.py:27: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(html)


,날짜,종가
0,2025-06-04,1489.0
1,2025-06-05,1626.0
2,2025-06-06,1633.0
3,2025-06-09,1691.0
4,2025-06-10,1680.0
5,2025-06-11,1738.0
6,2025-06-12,1904.0
7,2025-06-13,1968.0
8,2025-06-16,1975.0
9,2025-06-17,1952.0


-> 최근 한 달꺼만 크롤링됨. 두 달치를 가져오려면 사이트에서 달력을 클릭하고 날짜를 옮겨야하는데 이거는 크롤링 코드 구현이 좀 어렵네요..

2. SCFI(상하이 컨테이너 운임 지수)

-> 인베스팅닷컴에서 제공 안하고 중국쪽이라 크롤링하기 어려워요

3. 클락슨 신조선가 지수

-> 얘도 유료 서비스로 제공되고 로그인하고 구독해야한다네요